# HARRY POTTER CLASSIFIER

In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn graphviz

In [ ]:
import pandas as pd
from prepare_training_data import prepare_splits, train_all_models, evaluate_all
from visualize import visualize_all, plot_class_distributions, plot_original_distribution, analyze_accuracy_vs_depth

In [ ]:
data_path = "Data/harry_potter_students.csv"
hp_df = pd.read_csv(data_path)
hp_df.shape

In [ ]:
feature = hp_df.drop(['House'], axis=1)
label = hp_df['House']

In [ ]:
plot_original_distribution(label, "House distribution on original dataset")

In [ ]:
split_portions = [(0.4, 0.6), (0.6, 0.4), (0.8, 0.2), (0.9, 0.1)]
data_sets =  prepare_splits(feature, label, split_portions)

In [ ]:
plot_class_distributions(data_sets, split_portions, "House distribution")

In [ ]:
# train data
clfs = train_all_models(data_sets)
# visualize the trees based on trained models of datasets (clfs)
visualize_all(clfs, 
              feature_names=feature.columns.tolist(), 
              class_names=["Hufflepuff", "Gryffindor", "Ravenclaw", "Slytherin"])

In [ ]:
evaluate_all(clfs=clfs, 
             datasets=data_sets, 
             class_names=["Hufflepuff", 
                          "Gryffindor", 
                          "Ravenclaw", 
                          "Slytherin"])

In [ ]:
result = analyze_accuracy_vs_depth( dataset=data_sets[2], 
                                    feature_names=feature.columns.tolist(), 
                                    class_names=["Hufflepuff", 
                                                 "Gryffindor", 
                                                 "Ravenclaw", 
                                                 "Slytherin"])
print(result)